In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_data = pd.read_csv("data\\ML-MATT-CompetitionQT2021_train.csv")
test_data = pd.read_csv("data\\ML-MATT-CompetitionQT2021_test.csv")

In [3]:
train_data.shape, test_data.shape

((36904, 1), (9158, 1))